## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
sys.path.insert(0,"../..")
from matplotlib import pyplot as plt

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,-21.21,-159.78,73.40,83,100,10.29,overcast clouds
1,1,New Norfolk,AU,-42.78,147.06,62.60,63,0,5.82,clear sky
2,2,Joao Pinheiro,BR,-17.74,-46.17,79.32,63,65,7.45,broken clouds
3,3,Saskylakh,RU,71.92,114.08,3.29,89,100,25.21,overcast clouds
4,4,Hanna,CA,51.63,-111.90,15.80,100,12,5.82,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Joao Pinheiro,BR,-17.74,-46.17,79.32,63,65,7.45,broken clouds
8,8,Robertsport,LR,6.75,-11.37,80.56,78,39,4.56,scattered clouds
9,9,Ulu Tiram,MY,1.60,103.82,78.80,88,20,1.12,moderate rain
12,12,Srikakulam,IN,18.30,83.90,75.58,60,0,7.81,clear sky
15,15,Victoria,HK,22.29,114.16,75.00,69,98,11.01,overcast clouds
25,25,Sao Filipe,CV,14.90,-24.50,81.84,70,66,8.95,broken clouds
30,30,Port-Gentil,GA,-0.72,8.78,84.20,70,40,10.29,scattered clouds
33,33,Bengkulu,ID,-3.80,102.27,78.17,83,100,1.59,heavy intensity rain
35,35,Saint-Pierre,RE,-21.34,55.48,80.60,69,20,23.04,few clouds
36,36,Salalah,OM,17.02,54.09,87.80,38,0,5.82,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                224
City                   224
Country                224
Lat                    224
Lng                    224
Max Temp               224
Humidity               224
Cloudiness             224
Wind Speed             224
Current Description    224
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Joao Pinheiro,BR,79.32,broken clouds,-17.74,-46.17,
8,Robertsport,LR,80.56,scattered clouds,6.75,-11.37,
9,Ulu Tiram,MY,78.80,moderate rain,1.60,103.82,
12,Srikakulam,IN,75.58,clear sky,18.30,83.90,
15,Victoria,HK,75.00,overcast clouds,22.29,114.16,
25,Sao Filipe,CV,81.84,broken clouds,14.90,-24.50,
30,Port-Gentil,GA,84.20,scattered clouds,-0.72,8.78,
33,Bengkulu,ID,78.17,heavy intensity rain,-3.80,102.27,
35,Saint-Pierre,RE,80.60,few clouds,-21.34,55.48,
36,Salalah,OM,87.80,clear sky,17.02,54.09,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.count()

City                   224
Country                224
Max Temp               224
Current Description    224
Lat                    224
Lng                    224
Hotel Name             224
dtype: int64

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [11]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Joao Pinheiro,BR,79.32,broken clouds,-17.74,-46.17,Hotel Santos
8,Robertsport,LR,80.56,scattered clouds,6.75,-11.37,Kwepunha Retreat
9,Ulu Tiram,MY,78.80,moderate rain,1.60,103.82,Dajie Avenue Hotel
12,Srikakulam,IN,75.58,clear sky,18.30,83.90,Sriram residency
15,Victoria,HK,75.00,overcast clouds,22.29,114.16,Mini Hotel Central


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))